In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc, math
import pickle
from collections import Counter

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_class = 2
ROWS, COLS = 224, 224
BATCHSIZE = 64
SEAL_CLASSES = ['seal_0', 'seal_1', 'seal_2',  'seal_3_4', 'seal_5_9', 'seal_10']
nb_perClass = int(BATCHSIZE / len(SEAL_CLASSES))
TRAIN_DIR = '../darknet/seals/JPEGImagesBlk'
TEST_DIR = '../darknet/seals/JPEGImagesTest'
num_class = len(SEAL_CLASSES)

In [3]:
# Data generator
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)

# Fine-tune Example
rfcnCVtmp = pd.read_pickle('../coords/rfcnmultiCV.pkl')
rfcnCVtmp.head(3)

,img,proba,x0,y0,x1,y1,seal
0,228_54,0.894,364.95,295.10,464.95,395.10,0
1,228_54,0.893,228.20,376.30,328.20,476.30,0
2,228_54,0.720,407.75,409.95,507.75,509.95,0


In [4]:
# Read in the previous preds 
dftmp = pd.concat([pd.read_csv('../coords/vggCVPreds2604_fold2.csv'),
                pd.read_csv('../coords/vggCVPreds2604_fold1.csv')])
dftmp.columns = ['img1', 'predSeal']


In [5]:

rfcnCV = pd.concat([rfcnCVtmp.reset_index(drop=True), dftmp.reset_index(drop=True)], axis=1)
rfcnCV = rfcnCV[rfcnCV['predSeal']>0.2]
Counter(rfcnCV.seal)

Counter({0: 6512,
         1: 12900,
         2: 10070,
         3: 6830,
         4: 4906,
         5: 3199,
         6: 2106,
         7: 1364,
         8: 937,
         9: 628,
         10: 417,
         11: 260,
         12: 179,
         13: 148,
         14: 94,
         15: 54,
         16: 40,
         17: 31,
         18: 22,
         19: 14,
         20: 15,
         21: 6,
         22: 2,
         23: 2,
         24: 1,
         25: 3})

In [6]:
rfcnCV['seal_cut'] = pd.cut(rfcnCV['seal'], bins = [-1,0,1,2,4,10,30])

In [7]:
# Make classes
rfcnCV['seals_0'] = np.where(rfcnCV['seal']==0, 1, 0)
rfcnCV['seals_1'] = np.where(rfcnCV['seal'] == 1, 1, 0)
rfcnCV['seals_2'] = np.where(rfcnCV['seal'] == 2, 1, 0)
rfcnCV['seals_3_4'] = np.where(rfcnCV['seal'].between(3,4), 1, 0)
rfcnCV['seals_5_10'] = np.where(rfcnCV['seal'].between(5,10), 1, 0)
rfcnCV['seals_11+'] = np.where(rfcnCV['seal']>10, 1, 0)
rfcnCV.head(5)

,img,proba,x0,y0,x1,y1,seal,img1,predSeal,seal_cut,seals_0,seals_1,seals_2,seals_3_4,seals_5_10,seals_11+
12,332_22,0.998,444.00,9.25,544.00,109.25,1,332_22,0.987439,"(0, 1]",0,1,0,0,0,0
13,332_22,0.925,261.45,195.15,361.45,295.15,1,332_22,0.999877,"(0, 1]",0,1,0,0,0,0
18,448_21,0.989,0.00,422.25,100.00,522.25,0,448_21,0.942756,"(-1, 0]",1,0,0,0,0,0
23,338_32,0.958,444.00,444.00,544.00,544.00,2,338_32,0.944517,"(1, 2]",0,0,1,0,0,0
24,338_32,0.917,444.00,4.40,544.00,104.40,2,338_32,0.994015,"(1, 2]",0,0,1,0,0,0


In [8]:
# Lets make our validation set
folds = [rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 != 0,
        rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 == 0]
rfcnCV.shape

(50740, 16)

In [9]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(SEAL_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby(['seal_cut'], as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(TRAIN_DIR, row[0])
            seal = row[6]
            bbox = row[2:6]
            cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i] = row[10:] # Add in all classes
            i += 1
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)
        #return (batch_x.transpose(0, 3, 1, 2), batch_y)

In [10]:
def testcv_generator(df, datagen = None, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(TRAIN_DIR, row[0]+'.jpg')
            bbox = row[2:6]
            cropped = load_img(image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            if datagen is not None: x = datagen.random_transform(x)            
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%50 == 0: print(batch_index)
        #return(batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))

In [11]:
for fold in range(2):
    print "Fold: " + str(fold+1)
    train_df = rfcnCV[~folds[fold]]
    test_df = rfcnCV[folds[fold]]
    valid_df = test_df[:1500]
    
    # validation_data (valid_x,valid_y)
    print "Part ... A"
    df_1 = valid_df
    l = valid_df.groupby('seal').size()
    nb_NoF_valid = math.ceil(l.sum()/10)
    valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
    valid_y = np.zeros((valid_df.shape[0], len(SEAL_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in valid_df.iterrows():
        if index % 300 == 0 : print index
        row = row.tolist()
        image_file = os.path.join(TRAIN_DIR, row[0])
        seal = row[6]
        bbox = row[2:6]
        cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
        x = np.asarray(cropped, dtype=K.floatx())
        x = preprocess_input(x)
        valid_x[i] = x
        valid_y[i] = row[10:]
        i += 1
    valid_x = valid_x.transpose(0, 3, 1, 2)
    valid_x.shape

    # Load our model
    print "Part ... B"
    nb_epoch = 5
    samples_per_epoch = 50000
    model = vgg_std16_model(ROWS, COLS, channel, num_class)

    # Start Fine-tuning
    print "Part ... C"
    model.fit_generator(train_generator(train_datagen, train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch, #50000,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    for layer in model.layers[10:]:
        layer.trainable = True
    model.optimizer.lr = 1e-4
    nb_epoch = 5
    print "Part ... D"
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    # Test preds save
    print "Part ... E"
    test_preds = model.predict_generator(testcv_generator(test_df), val_samples=test_df.shape[0])
    df = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(test_preds,  columns=SEAL_CLASSES)], axis=1)
    df.to_pickle('../coords/vggCVMultiPreds1805_fold' + str(fold+1) + '.pkl')
    df[[0]+range(15, 20)].to_csv('../coords/vggCVMultiPreds1805_fold' + str(fold+1) + '.csv', index=False)
    
    # Clean up
    del model, train_df, test_df, valid_df, valid_x, df_1
    gc.collect()
    gc.collect()

Fold: 1
Part ... A
72300
74400
75300
Part ... B
Part ... C
Epoch 1/5
49984/50000 [============================>.] - ETA: 0s - loss: 0.9741 - acc: 0.5907

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


50048/50000 [==============================] - 2988s - loss: 0.9740 - acc: 0.5908 - val_loss: 1.0264 - val_acc: 0.5460
Epoch 2/5
50048/50000 [==============================] - 2986s - loss: 0.6766 - acc: 0.7171 - val_loss: 1.0033 - val_acc: 0.5673
Epoch 3/5
50048/50000 [==============================] - 2985s - loss: 0.5685 - acc: 0.7653 - val_loss: 0.9725 - val_acc: 0.6140
Epoch 4/5
50048/50000 [==============================] - 2985s - loss: 0.4910 - acc: 0.7999 - val_loss: 1.2319 - val_acc: 0.5613
Epoch 5/5
50048/50000 [==============================] - 2985s - loss: 0.4223 - acc: 0.8291 - val_loss: 1.1233 - val_acc: 0.5967
Part ... D
Epoch 1/5
50048/50000 [==============================] - 2985s - loss: 0.3646 - acc: 0.8551 - val_loss: 1.1721 - val_acc: 0.5933
Epoch 2/5
50048/50000 [==============================] - 2983s - loss: 0.2981 - acc: 0.8850 - val_loss: 1.2333 - val_acc: 0.6100
Epoch 3/5
50048/50000 [==============================] - 2983s - loss: 0.2522 - acc: 0.9042 - va